# MAL With Annotation Types
* Image MAL with subclasses.
* This is the same task as the image mal tutorial but we are going to add a subclass for whether or not the person in the image is holding a bag.

In [1]:
try:
    import labelbox
except: 
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python && git checkout ms/annotation-examples && pip install .[data]

In [2]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !mv labelbox-python/examples/model_assisted_labeling/image_model.py .
else:
    import sys
    sys.path.append('../model_assisted_labeling')

In [3]:
#Used this as a reference for the model
#https://colab.research.google.com/github/tensorflow/tpu/blob/master/models/official/mask_rcnn/mask_rcnn_demo.ipynb#scrollTo=6lCL-ZcwaJbA
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, LabelingFrontend
from labelbox.data.annotation_types import (
    LabelList,
    RasterData,
    Rectangle,
    ObjectAnnotation,
    ClassificationAnnotation,
    Point,
    ClassificationAnswer,
    Radio,
    Mask,
    Label
)
from labelbox.data.serialization import NDJsonConverter
from image_model import predict, class_mappings, load_model
from typing import Dict, Any, Tuple, List
import numpy as np
from PIL import Image
import requests
import ndjson
import uuid
from io import BytesIO
import os
from getpass import getpass

In [4]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [5]:
# Set this if running in colab. Otherwise it should work if you have the LABELBOX_API_KEY set.
API_KEY = os.environ["LABELBOX_API_KEY"]
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

In [6]:
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

In [7]:
#Downloads weights and loads the model.
load_model()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://cloud-tpu-checkpoints/mask-rcnn/1555659850/variables/variables


## Create Predictions
* Create helper functions for processing the model outputs
* Make predictions
* Add predictions to a LabelList object

In [10]:
def has_bag(person, bags):
    for bag in bags:
        if person.value.shapely.contains(bag.value.shapely.centroid):
            return True
    return False    

def get_annotations(boxes, classes, seg_masks):
    annotations = []
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        name = class_mappings[class_idx]
        value = None
        classifications = []
        if name in ['person', 'handbag']:
            value = Rectangle(
                start = Point(x = box[1], y = box[0]), end = Point(x = box[3], y = box[2])
            )
        elif name == 'car':
            value = Mask(mask = RasterData.from_2D_arr(arr = seg), color = (1,1,1))
        if value is not None:
            annotations.append(
                ObjectAnnotation(
                    name = name,
                    value = value
                )
            ) 
    return annotations

def update_bag_classifications(annotations):
    bags = [annot for annot in annotations if annot.name == 'handbag']
    people = [annot for annot in annotations if annot.name == 'person']
    for person in people:
        person.classifications = [ClassificationAnnotation(
            name = 'has_bag',
            value = Radio(answer = ClassificationAnswer(name = str(has_bag(person, bags))))
        )]


In [11]:
### We can just start creating predictions whether or not we have a 
image_urls = ['https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg']

labellist = LabelList([])

for image_url in image_urls:
    image_data = RasterData(url = image_url)
    height, width = image_data.data.shape[:2]
    prediction = predict(np.array([image_data.im_bytes]), min_score=0.5, height=height, width = width)
    annotations = get_annotations(prediction['boxes'], prediction['class_indices'], prediction['seg_masks'])
    update_bag_classifications(annotations)
    labellist.append(Label(
        data = image_data,
        annotations = annotations
    ))

## Project Setup
* Create project
* Use labellist.get_ontology() to automatically create the OntologyBuilder

In [12]:
# Lets setup a project to label
# Note see Ontology, Project, and Project_setup notebooks for more information on this section.
project = client.create_project(name="mal_project")
dataset = client.create_dataset(name="mal_dataset")
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == 'editor'))
# Use the label collection to build the ontology
project.setup(editor, labellist.get_ontology().asdict())
project.datasets.connect(dataset)
project.enable_model_assisted_labeling()

True

## Add ids required for MAL
* Use helper functions to add urls to images and seg masks, assign schema ids to features, and add all data rows to the dataset.

In [13]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
labellist.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .assign_schema_ids(OntologyBuilder.from_project(project)) \
         .add_to_dataset(dataset, signer)

1it [00:02,  2.18s/it]
1it [00:00, 4549.14it/s]
1it [00:00, 13148.29it/s]


## Convert to Prediction import format (NDJson)
* We want to create a json payload that matches this: https://docs.labelbox.com/data-model/en/index-en#annotations
* We can use the NDJsonConverter to turn our labellist containg predictions into ndjson

In [14]:
ndjsons = list(NDJsonConverter.serialize(labellist))
print(ndjsons[0])

{'uuid': '12947ab8-616f-4493-a310-d04e95adabca', 'dataRow': {'id': 'ckrmse31q7vvy0yu0bac35o1z'}, 'schemaId': 'ckrmsae6301e00y8dejvfc7ar', 'classifications': [{'schemaId': 'ckrmsae6o01e60y8d7o0e3j9x', 'answer': {'schemaId': 'ckrmsae7801e80y8ddmar36r7'}}], 'bbox': {'top': 1352.3682861328125, 'left': 2275.82861328125, 'height': 350.1317138671875, 'width': 139.7919921875}}


### Upload the annotations

In [15]:
upload_task = project.upload_annotations(name=f"upload-job-{uuid.uuid4()}",
                                         annotations=ndjsons,
                                         validate=True)
# Wait for upload to finish
upload_task.wait_until_done()

In [16]:
# Review the upload status
for status in upload_task.statuses:
    print(status)

{'uuid': '12947ab8-616f-4493-a310-d04e95adabca', 'dataRow': {'id': 'ckrmse31q7vvy0yu0bac35o1z'}, 'status': 'SUCCESS'}
{'uuid': '864a560f-0b53-446b-83cd-c9fff1586495', 'dataRow': {'id': 'ckrmse31q7vvy0yu0bac35o1z'}, 'status': 'SUCCESS'}
{'uuid': 'e08f528f-3cf6-4bd8-8468-09b18c3380fb', 'dataRow': {'id': 'ckrmse31q7vvy0yu0bac35o1z'}, 'status': 'SUCCESS'}
{'uuid': '96c30b98-323e-4aee-8cfa-626c727d1b19', 'dataRow': {'id': 'ckrmse31q7vvy0yu0bac35o1z'}, 'status': 'SUCCESS'}
{'uuid': 'ef570524-80de-4765-972f-f7caf8b866d1', 'dataRow': {'id': 'ckrmse31q7vvy0yu0bac35o1z'}, 'status': 'SUCCESS'}
{'uuid': '2c9aa3a0-26a6-474e-99d6-60cd23a86b77', 'dataRow': {'id': 'ckrmse31q7vvy0yu0bac35o1z'}, 'status': 'SUCCESS'}
{'uuid': 'f860861f-2cb4-48dc-9314-1871138a44f0', 'dataRow': {'id': 'ckrmse31q7vvy0yu0bac35o1z'}, 'status': 'SUCCESS'}
{'uuid': '5a584ef4-8005-479e-8677-def103944ebe', 'dataRow': {'id': 'ckrmse31q7vvy0yu0bac35o1z'}, 'status': 'SUCCESS'}
{'uuid': '8901ecd8-ee62-4fb1-8b83-e63e01ce3b09', 'dataRo